In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Librerías para preprocesado de datos
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Modelos de clasificación y regresión para selección de características y entrenamiento
from sklearn.ensemble import RandomForestClassifier

# Librerías para tratamiento de datos desbalanceados
from imblearn.over_sampling import SMOTE

# Librerías para división de los datos
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

In [33]:
df = pd.read_csv("./data/train_data.csv")
df_label = pd.read_csv("./data/train_labels.csv")
df_test = pd.read_csv("./data/test_data.csv")

In [34]:
# Obtenemos el porcentaje de nulos de cada columna
porcentaje_nulos = df.isnull().mean() * 100

# Nos quedamos con el subconjunto de columnas que tengan porcentaje de nulos superior
# al 50%
porcentaje_nulos = porcentaje_nulos[porcentaje_nulos > 50]

# Eliminamos del Dataframe original las columnas que coincidan con las seleccionadas
# en `porcentaje_nulos`
df = df.drop(columns=porcentaje_nulos.index)
df_test = df_test.drop(columns=porcentaje_nulos.index)

In [35]:
# Seleccionamos las columnas de tipo numérico del DataFrame `df_completo`.
numerical_columns = df.select_dtypes(include=['number']).columns

# Configuramos el imputer para usar la estrategia de imputación de la media.
imputer = SimpleImputer(strategy='mean')

# Aplicamos la transformación de imputación a las columnas numéricas seleccionadas.
# Aquí usamos `fit_transform` para calcular y aplicar la media de cada columna, rellenando los valores nulos con esta media.
df[numerical_columns] = imputer.fit_transform(df[numerical_columns])
df_test[numerical_columns] = imputer.fit_transform(df_test[numerical_columns])

In [36]:
columnas_objeto = df.columns[df.dtypes == 'object']

# Creamos la array para el mapeo
frequency_map = {
    'very_low': 0,
    'low': 1,
    'moderate_low': 2,
    'moderate': 3,
    'moderate_high': 4,
    'high': 5,
    'very_high': 6
}
# Crear el imputador con la estrategia de moda
imputer = SimpleImputer(strategy='most_frequent')
for col in ['Infraction_CLH', 'Base_67254', 'Infraction_TEN']:
    df[col] = df[col].map(frequency_map)
    df[col] = imputer.fit_transform(df[[col]])
    df_test[col] = df_test[col].map(frequency_map)
    df_test[col] = imputer.fit_transform(df_test[[col]])

# Rellenamos valores nulos en 'Infraction_DQLY' con la moda de la columna
# Esto asegura que no queden valores faltantes antes de aplicar One-Hot Encoding
df['Infraction_DQLY'] = df['Infraction_DQLY'].fillna(df['Infraction_DQLY'].mode())
df_test['Infraction_DQLY'] = df_test['Infraction_DQLY'].fillna(df_test['Infraction_DQLY'].mode())

# Definimos las columnas que queremos transformar usando One-Hot Encoding
columns_to_encode = ['Infraction_YFSG', 'Infraction_DQLY']

# Aplicamos pd.get_dummies() a las columnas categóricas seleccionadas
# Esto crea una columna binaria para cada categoría única en estas columnas
df = pd.get_dummies(df, columns=columns_to_encode)
df_test = pd.get_dummies(df_test, columns=columns_to_encode)

In [37]:
# Seleccionamos las columnas de tipo booleano en el DataFrame
boolean_columns = df.select_dtypes('bool').columns

# Iteramos sobre cada columna booleana para manejar valores nulos y convertir el tipo de datos
for col in boolean_columns:
        # Completamos los valores nulos de la columna con el valor más frecuente
        # Esto se logra utilizando `value_counts().head(1)` para obtener el valor más frecuente (la moda)
        df[col] = df[col].fillna(df[col].value_counts().head(1))

        # Convertimos la columna booleana a tipo entero (0 y 1 en lugar de True y False)
        df[col] = df[col].astype(int)

        df_test[col] = df_test[col].fillna(df_test[col].value_counts().head(1))
        df_test[col] = df_test[col].astype(int)

In [38]:
# Convertimos la columna 'Expenditure_AHF' a formato de fecha y hora (DateTime)
df['Expenditure_AHF'] = pd.to_datetime(df['Expenditure_AHF'])

# Convertimos los datos de fecha a segundos desde el inicio de la época (epoch) y los escalamos a una unidad más manejable
df['Expenditure_AHF'] = df['Expenditure_AHF'].astype(int) / 10**9

df_test['Expenditure_AHF'] = pd.to_datetime(df_test['Expenditure_AHF'])
df_test['Expenditure_AHF'] = df_test['Expenditure_AHF'].astype(int) / 10**9

In [39]:
df = df.groupby('ID', as_index=False).sum()
df_test = df_test.groupby('ID', as_index=False).sum()

In [40]:
# Creamos una instancia del StandardScaler para estandarizar los datos
scaler = StandardScaler()

# Seleccionamos las columnas numéricas del DataFrame
numerical_columns = df.select_dtypes(include=['number']).columns

# Aplicamos el escalador a todas las columnas numéricas
# El método fit_transform calcula la media y desviación estándar de cada columna y transforma los datos
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
df_test[numerical_columns] = scaler.fit_transform(df_test[numerical_columns])

In [41]:
#igualamos La variable X al dataframe base
df = df.drop('ID', axis=1)
df_test = df_test.drop('ID', axis=1)

# Almacenamos en la variable las etiquestas sin el `ID`
y = df_label.drop('ID', axis=1)
df_label = df_label.drop('ID', axis=1)

In [42]:
X = df[['Expenditure_AHF', 'Expenditure_JIG', 'Base_02683', 'Base_6872',
       'Infraction_GGO', 'Risk_9995', 'Infraction_AYWV', 'Expenditure_YTR',
       'Infraction_PAS', 'Base_14808', 'Infraction_TBP', 'Base_0580',
       'Expenditure_RGD', 'Base_0229', 'Risk_1475', 'Base_8511',
       'Infraction_JYZB', 'Base_22178', 'Risk_7095', 'Base_36516',
       'Infraction_HFU', 'Expenditure_HRQ', 'Risk_2102', 'Risk_4804',
       'Infraction_XWX', 'Expenditure_XDD', 'Risk_8742', 'Infraction_VHU',
       'Risk_4247', 'Infraction_GSS', 'Risk_0454', 'Expenditure_HKXV',
       'Infraction_MHM', 'Risk_4160', 'Risk_9367', 'Base_7910',
       'Expenditure_GMC', 'Risk_9423', 'Risk_6178', 'Risk_6197',
       'Infraction_QVSL', 'Infraction_QXUM', 'Risk_8532', 'Risk_9247',
       'Expenditure_ONEG', 'Expenditure_MTRQ', 'Expenditure_HPM',
       'Infraction_HFSI', 'Base_5441', 'Risk_8902', 'Infraction_PTY',
       'Expenditure_BEH', 'Infraction_LMHK', 'Infraction_NMCB',
       'Infraction_TPAF', 'Infraction_ZRH', 'Infraction_WIS', 'Infraction_IIZ',
       'Infraction_WVAW', 'Infraction_KEJT', 'Infraction_WMAQ',
       'Infraction_SIA', 'Infraction_CZXL', 'Infraction_HUK',
       'Infraction_LIES', 'Infraction_LSX', 'Infraction_DNOU',
       'Infraction_YFSG_CL', 'Infraction_YFSG_XL', 'Infraction_YFSG_XM']]

X_test = df_test[['Expenditure_AHF', 'Expenditure_JIG', 'Base_02683', 'Base_6872',
       'Infraction_GGO', 'Risk_9995', 'Infraction_AYWV', 'Expenditure_YTR',
       'Infraction_PAS', 'Base_14808', 'Infraction_TBP', 'Base_0580',
       'Expenditure_RGD', 'Base_0229', 'Risk_1475', 'Base_8511',
       'Infraction_JYZB', 'Base_22178', 'Risk_7095', 'Base_36516',
       'Infraction_HFU', 'Expenditure_HRQ', 'Risk_2102', 'Risk_4804',
       'Infraction_XWX', 'Expenditure_XDD', 'Risk_8742', 'Infraction_VHU',
       'Risk_4247', 'Infraction_GSS', 'Risk_0454', 'Expenditure_HKXV',
       'Infraction_MHM', 'Risk_4160', 'Risk_9367', 'Base_7910',
       'Expenditure_GMC', 'Risk_9423', 'Risk_6178', 'Risk_6197',
       'Infraction_QVSL', 'Infraction_QXUM', 'Risk_8532', 'Risk_9247',
       'Expenditure_ONEG', 'Expenditure_MTRQ', 'Expenditure_HPM',
       'Infraction_HFSI', 'Base_5441', 'Risk_8902', 'Infraction_PTY',
       'Expenditure_BEH', 'Infraction_LMHK', 'Infraction_NMCB',
       'Infraction_TPAF', 'Infraction_ZRH', 'Infraction_WIS', 'Infraction_IIZ',
       'Infraction_WVAW', 'Infraction_KEJT', 'Infraction_WMAQ',
       'Infraction_SIA', 'Infraction_CZXL', 'Infraction_HUK',
       'Infraction_LIES', 'Infraction_LSX', 'Infraction_DNOU',
       'Infraction_YFSG_CL', 'Infraction_YFSG_XL', 'Infraction_YFSG_XM']]

y = df_label['label']

In [43]:
# realizar sobremuestreo en la clase minoritaria. Usamos random_state=42 para asegurar
# que los resultados sean reproducibles en cada ejecución.
smote = SMOTE(random_state=42)

# Aplicamos SMOTE a las características X y las etiquetas y.
# Esto genera nuevas muestras sintéticas de la clase minoritaria, devolviendo un nuevo
# conjunto de datos balanceado en X_resampled e y_resampled.
X_resampled, y_resampled = smote.fit_resample(X, y)

In [44]:
rf = RandomForestClassifier(n_estimators=175, random_state=42)
rf.fit(X, y)

y_train_pred = rf.predict(X)
f1_train = f1_score(y, y_train_pred)  # Para problemas de clasificación
print(f'F1-score en entrenamiento: {f1_train}')

# Evaluar el rendimiento en el conjunto de prueba
y_test_pred = rf.predict(X_test)
y_test_pred_df = pd.DataFrame(y_test_pred, columns=['label'])
y_test_pred_df.to_csv("./test_labels.csv", index=False)


F1-score en entrenamiento: 1.0


In [ ]:
print(y_test_pred_df["label"].unique())

   label
0      0
1      0
2      0
3      0
4      0
